# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
file_path_list = glob.glob(os.path.join(root,'*'))
#print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = {
        'class' : 'SimpleStrategy', 
        'replication_factor' : 1
    }
""")

#### Set Keyspace

In [7]:
session.set_keyspace("sparkifydb")

## Task 1: Getting the artist, song title and length for a given session Id and Item in session

For this query the `session_id` and `item_in_session` are considered as the composite key because we are applying the `WHERE` clause directly onto those fields.

In [8]:
query = """
    CREATE TABLE IF NOT EXISTS song_plays (
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        song_length float,
        PRIMARY KEY ((session_id, item_in_session))
    )
"""
session.execute(query)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
            INSERT INTO song_plays (
                session_id, 
                item_in_session, 
                artist, 
                song_title, 
                song_length
            )
            VALUES (%s, %s, %s, %s, %s)
        """
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
query = """
    SELECT artist, song_title, song_length FROM song_plays
    WHERE session_id = 338 AND item_in_session = 4
"""

rows = session.execute(query)
for row in rows:
    print(f"{row.artist}, {row.song_title}, {row.song_length}")

Faithless, Music Matters (Mark Knight Dub), 495.30731201171875


## Task 2: Getting artist, song title and user information for a given user Id and session Id

For this query, the `user_id` and `session_id` are considered as a composite partition key to speed up the reads because we need to apply a `WHERE` clause on those fields. To store the data sorted by `item_in_session` we make that our cluster column (this is also to prevent overwriting multiple entries for a given user in the same session).

In [11]:
query = """
    CREATE TABLE IF NOT EXISTS plays_per_user (
        user_id int,
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        first_name text,
        last_name text,
        PRIMARY KEY ((user_id, session_id), item_in_session)
    )
"""
session.execute(query)

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """
            INSERT INTO plays_per_user (
                user_id,
                session_id, 
                item_in_session,
                artist, 
                song_title, 
                first_name, 
                last_name
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
query = """
    SELECT artist, song_title, first_name, last_name FROM plays_per_user
    WHERE user_id = 10 AND session_id = 182
"""

rows = session.execute(query)
for row in rows:
    print(f"{row.artist}, {row.song_title}, {row.first_name}, {row.last_name}")

Down To The Bone, Keep On Keepin' On, Sylvie, Cruz
Three Drives, Greece 2000, Sylvie, Cruz
Sebastien Tellier, Kilometer, Sylvie, Cruz
Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie, Cruz


## Task 3: Getting the user name given a song title

For this query we use the `song_title` as the partition key to perform the indexing for the `WHERE` clause, and use `user_id` as the clustering column to avoid overwriting entries for users with the same name.

In [14]:
query = """
    CREATE TABLE IF NOT EXISTS users_per_song (
        song_title text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY (song_title, user_id)
    )
"""
session.execute(query)

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """
            INSERT INTO users_per_song (
                song_title,
                user_id,
                first_name,
                last_name
            )
            VALUES (%s, %s, %s, %s)
        """
        
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
query = """
    SELECT first_name, last_name FROM users_per_song
    WHERE song_title = 'All Hands Against His Own'
"""

rows = session.execute(query)
for row in rows:
    print(f"{row.first_name}, {row.last_name}")

Jacqueline, Lynch
Tegan, Levine
Sara, Johnson


### Drop the tables before closing out the sessions

In [17]:
query = """
    DROP TABLE IF EXISTS song_plays
"""
session.execute(query)

query1 = """
    DROP TABLE IF EXISTS plays_per_user
"""
session.execute(query1)

query2 = """
    DROP TABLE IF EXISTS users_per_song
"""
session.execute(query2)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()